# cifar-10 cnn

## imports

In [22]:
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder

## data

In [2]:
def unpickle(file):
    with open(file, "rb") as f:
        data_map = pickle.load(f, encoding="bytes")
    return data_map

In [3]:
data = unpickle("../my_datasets/cifar-10-batches-py/data_batch_1")

In [4]:
#10000 images, 1024 pixels (and 3 color channels, 1024 * 3)
data[b"data"].shape

(10000, 3072)

In [5]:
type(data[b"data"])

numpy.ndarray

In [6]:
num_images = data[b"data"].shape[0]
num_pixels = int(data[b"data"].shape[1] / 3)
num_colors = 3
img_dim = [int(num_pixels ** .5)] * 2
print("images: {}\ndimensions: {}\npixels: {}\ncolors: {}".format(
    num_images, img_dim, num_pixels, num_colors
))

images: 10000
dimensions: [32, 32]
pixels: 1024
colors: 3


### reshaping the data
We'll start by reshaping the 2nd dimension of each image to be of shape (3, 1024), where all red features are (0, ?), all green are (1, ?) and all blue are (2, ?). The shape of the entire dataset should afterwards be (10000, 1024, 3).

In [34]:
features = np.asarray(
    list(
        map(
            lambda img: [
                [img[i], img[num_pixels + i], img[2*num_pixels + i]] for i in range(num_pixels)
            ],
            data[b"data"]
        )
    )
)
#).reshape(num_images, img_dim[0], img_dim[1], num_colors)

In [35]:
features.shape

(10000, 1024, 3)

In [14]:
#plt.imshow(reshaped_data[9])

## labels
Below we'll one-hot encode the labels.

In [30]:
labels = np.asarray(data[b"labels"])
num_classes = np.unique(labels).shape[0]
num_classes

10

In [31]:
labels.shape

(10000,)

In [32]:
#one-hot encoding
y_true = OneHotEncoder().fit_transform(labels.reshape(-1, 1)).toarray()
y_true[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

## network setup
Below we'll setup the network. We'll create a 2 layer convolution (each with a max pooling layer) connected to 2 dense layers. This model is based on the TensorFlow [CNN tutorial](https://www.tensorflow.org/tutorials/layers).

placeholders

In [33]:
x = tf.placeholder(dtype=tf.float32, shape=[None, num_pixels])
y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

input layer

In [48]:
input_layer = tf.reshape(x, [-1, img_dim[0], img_dim[1], num_colors])

### convolution parameters

In [49]:
#convolution
filters = [32, 64]
kernel_sizes = [[5, 5], [5, 5]]
paddings = ["same", "same"]
activations = [tf.nn.relu, tf.nn.relu]

#pooling
pool_sizes = [[2, 2], [2, 2]]
strides = [2, 2]

convolution 0

In [51]:
#convolution
conv0 = tf.layers.conv2d(
    inputs=input_layer,
    filters=filters[0],
    kernel_size=kernel_sizes[0],
    padding=paddings[0],
    activation=activations[0]
)
#pool
pool0 = tf.layers.max_pooling2d(
    inputs=conv0,
    pool_size=pool_sizes[0],
    strides=strides[0]
)

convolution 1

Notice that with this structure, the convolutional portion of the network is easily refactorable into a loop.

In [52]:
#convolution
conv1 = tf.layers.conv2d(
    inputs=pool0,
    filters=filters[1],
    kernel_size=kernel_sizes[1],
    padding=paddings[1],
    activation=activations[1]
)
#pool
pool1 = tf.layers.max_pooling2d(
    inputs=conv1,
    pool_size=pool_sizes[1],
    strides=strides[1]
)

flatten

In [58]:
resulting_img_dim = img_dim[:]
for p in pool_sizes:
    for j in range(2):
        resulting_img_dim[j] = int(resulting_img_dim[j] / p[j])
resulting_img_dim

[8, 8]

In [59]:
conv_flat = tf.reshape(tensor=pool1, shape=[-1, resulting_img_dim[0] * resulting_img_dim[1] * filters[1]])

dense 0

In [61]:
#seems like units choice is open to interpretation
drop_rate = tf.placeholder(tf.float32, shape=[1])
dense0 = tf.layers.dense(inputs=conv_flat, units=1024, activation=tf.nn.relu)
dropout0 = tf.layers.dropout(inputs=dense0, rate=drop_rate)

logits (y probabilities)

In [62]:
logits = tf.layers.dense(inputs=dropout0, units=num_classes)

### loss

In [63]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    labels=labels,
    logits=logits
))

## hyperparams

In [64]:
learning_rate = 0.001
epochs = 300
batch_size = 16
dropout_prob_train = 0.5
dropout_prob_test = 1.0
epochs_between_output = 20

## optimizer

In [ ]:
optimizer = tf.trace